In [1]:
from model import HotspotPredictor
import pandas as pd
import numpy as np
import mysql.connector as msc
import datetime as dt
from collections import Counter
from bokeh.io import output_file, show, output_notebook
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)
from sklearn.cluster import KMeans
import random
from sklearn.ensemble import RandomForestRegressor
from geopy.distance import vincenty
from sklearn.cross_validation import train_test_split, cross_val_score

/Users/William/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [42]:
engine = msc.connect(user='root', password=<PASSWORD>,
                              host='127.0.0.1',
                              database='rideaustin')
df = pd.read_sql(
    'SELECT start_location_lat,start_location_long, created_date,tod, status, total_fare \
    FROM rides  \
    WHERE status = "NO_AVAILABLE_DRIVER" \
    OR status = "Completed";', engine)

In [525]:
hotspot1 = HotspotPredictor(d1)

model.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.df['day_of_week'] = self.df['created_date'].dt.weekday


In [529]:
df_preds = hotspot1.df
rides = hotspot.predict_n_rides(df_preds,datetime=pd.to_datetime('2016-10-09 22:00:00'))
rides

17

In [5]:
errors = hotspot.create_random_tests(df=df,n_repititions=10)

model.py:305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  new_row['period'] = new_row['period']+1


completed error! 1.75675336296
completed error! 1.55593099213
completed error! 1.27823656892
completed error! 1.03934097286
completed error! 1.37177800017


In [486]:
# df_birthday = df[df['created_date'].dt.date == dt.date(2016,10,9)]
# rides = hotspot.predict_rides(d1)
d1['period'] = d1.apply(hotspot.period,axis=1)

/Users/William/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [502]:
rides = hotspot.predict_n_rides(d1,datetime=d1.tail(1)['created_date'])

In [504]:
hotspot.predict_rides(d1)

'not enough data to predict hotspots'

In [485]:
d1,d2 = hotspot.split_at_date(df,pd.to_datetime('2016-10-09 22:00:00'))
cents, cent_dict = hotspot.create_centroids(d1)

In [530]:
def predict_rides(input_dataframe):
    '''
    Takes in the most recent ride request and predicts the next half-hour worth of ride quests using
    find_similar_pairs method.

    Inputs:
    dataframe,
    row,
    n_rides (to be predicted in separate function)

    output:
    Dataframe (predicted ride requests)'''

    predicted= []
    input_dataframe = input_dataframe[['start_location_lat','start_location_long','created_date','tod','day_of_week','period']]
    last_ride = input_dataframe.tail(1)
    datetime = last_ride['created_date']
    last_ride = last_ride.values[0]
    values = input_dataframe.values
    n_rides = predict_n_rides(input_dataframe)
    if n_rides < 10:
        return "not enough data to predict hotspots"
    for rep in xrange(n_rides):
        distribution = find_similar_pairs(values,last_ride)
        try:
            sample = distribution.sample().values
        except:
            return "not enough data to predict hotspots"
        if len(sample) < 5:
            sample = sample[0]
        predicted.append(sample)
        last_ride=sample
    return pd.DataFrame(predicted, columns=['start_location_lat','start_location_long','created_date',\
                                            'tod','day_of_week','period'])


In [515]:
def find_similar_pairs(dataframe,ride_request):
    '''
    Takes in a dataframe to test against and a ride request. Populates a distribution of other rides that
    could follow that particular ride. 

     
    input:
    -original dataframe.values (ndarray)
    -row (ndarray)
    output:
    -list of possible new points to sample from (Dataframe)
    '''

#     row2 = row2[['period','day_of_week', 'start_location_lat','start_location_long']].values
    dataframe = dataframe
    following_rides_list = []
    row1 = ride_request
    for i, request in enumerate(dataframe):
        if row1[5] <= request[5] <= row1[5]+1 and row1[4] == request[4] \
        and row1[0] -.004 <= request[0] <= row1[0]+.004 \
        and row1[1]-.004 <= request[1] <= row1[1]+.004:
#             request2 = dataframe[ind+1]
#             if row2[0]-1 <= request2[0] <= row2[0]+1:# and row2[1] == request2[1] \
#             #and row2[2]-.006 <= request2[2] <= row2[2]+.006\
#             #and row2[3]-.006 <= request2[3] <= row2[3]+.006:
            try:
                request2 = dataframe[i+1]
            except:
                continue
            following_rides_list.append(request2)
    return pd.DataFrame(following_rides_list, columns=['start_location_lat','start_location_long','created_date',\
                                            'tod','day_of_week','period'])

In [516]:
def prep_df(dataframe):
    '''
    takes in dataframe of raw data and cleans it so that it can be passed into my model to predict n_rides for
    the next half-hour block.
    
    input:
    -dataframe
    
    output:
    -X dataframe
    -y dataframe'''
    data = dataframe.copy()
    data['month']= data['created_date'].dt.month
    data['day'] = data['created_date'].dt.day
    data['day_of_week'] = data['created_date'].dt.weekday
    # creating a COUNT response variable
    data = data.groupby(['day_of_week','period','day'])['start_location_lat'].count().reset_index(name="count")
    #data = pd.get_dummies(data, columns=['day_of_week'], drop_first = True)
    y = data.pop('count')
    X = data
    return X,y

In [517]:
def predict_n_rides(data):
    '''
    takes in data and a split datetime and predicts how many n_rides there will be for the next half hour.
    
    input:
    - data (dataframe)
    - datime (string or datetime obj)
    
    output:
    -int'''
    data,response = prep_df(data)
    rf1 = RandomForestRegressor(n_estimators=100)
    model = rf1.fit(data,response)
    new_row = data.tail(1)
    new_row['period'] = new_row['period']+1
    n_rides = model.predict(new_row).astype(int)[0] #return prediction as integer
    if n_rides <= 10:
        print "Error: Not enough ride-traffic to predict hotspots."
    return n_rides

In [538]:
d1 = d1[(d1['start_location_lat'] >= 30.190833) & (d1['start_location_lat'] <= 30.404041)]
d1 = d1[(d1['start_location_long'] >=-97.819014) & (d1['start_location_long'] <= -97.647192)]

In [539]:
rides = predict_rides(d1)

/Users/William/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [540]:
centroids, centroid_dict = hotspot.create_centroids(rides)

In [542]:
pd.DataFrame(centroids,columns=['lat','long']).to_csv('future.csv')

In [453]:
previous_hotspots = pd.DataFrame(cents[:,0], columns=['lat'])
previous_hotspots['long']= cents[:,1]
d11 = d1[d1['total_fare']!=0.00]
previous_hotspots['avg_fare'] = d11.groupby(['cluster'])['total_fare'].mean()
previous_hotspots['percent_fulfilled'] = d1.groupby(['cluster'])['status_value'].mean()

In [467]:
previous_hotspots['requests'] = d1.groupby(['cluster'])['start_location_lat'].count()

In [469]:
previous_hotspots.to_csv('previous_hotspots.csv')

In [471]:
model = KMeans(n_clusters=5)
model.fit(np.array(d2[['start_location_lat','start_location_long']]))
centroident = model.cluster_centers_
d2['cluster'] = model.labels_

/Users/William/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [91]:
plot_Austin_centroids(cents,num_datapoints= len(d1),centroid_dictionary=cent_dict, unfulfilled_rides=d1)

In [77]:
d1['period'] = d1.apply(hotspot.period,axis=1)

/Users/William/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [79]:
d1 = d1[d1['period']>=42]

In [111]:
b_loon = {'COMPLETED':1,'NO_DRIVER_AVAILABLE':0}
d1['status_value'] = d1['status'].map(b_loon)

/Users/William/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [93]:
d1['cluster'].unique()

array([0, 2, 3, 4, 1])

cluster
0    12.439747
1    20.069167
2    12.284731
3    13.031343
4    16.499231
Name: total_fare, dtype: float64

cluster
0    745
1     93
2    184
3    358
4    126
Name: start_location_lat, dtype: int64

In [116]:
d1 = d1.fillna(value=0)

In [119]:
dd = pd.DataFrame(d1.groupby(['cluster'])['status_value'].mean())

In [120]:
dd['total_rides'] = d1.groupby(['cluster'])['start_location_lat'].count()
dd['avg_fare'] = d1.groupby(['cluster'], as_index=True)['total_fare'].mean()

In [121]:
dd['lat'] = 

,status_value,total_rides,avg_fare
cluster,,,
0,0.318121,745,3.957342
1,0.129032,93,2.589570
2,0.505435,184,6.209130
3,0.603352,358,7.862486
4,0.309524,126,5.106905


In [73]:
def plot_Austin_centroids(centroids, centroid_dictionary,num_datapoints, completed_rides=None, unfulfilled_rides=None):
    '''
    Takes in centroid values from create_centroids() and centroid_dictionary and plots the centroids relative to their
    intensity. Optional inputs for the lat-long columns for completed_rides (green) and unfulfilled_rides(blue).
    
    INPUT:
    - centroids (numpy array)
    - centroid_dict (dictionary)
    - copmleted_rides (dataframe)
    - unfulfilled_rides (dataframe)
    
    OUTPUT:
    -None
    '''
    #creating the plot
    map_options = GMapOptions(lat=30.29, lng=-97.73, map_type="roadmap", zoom=11)

    plot = GMapPlot(
        x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options
    )
    plot.title.text = "Austin"
    plot.api_key = <GOOGLE API KEY>
    
    #create alpha based on intensity
    alpha = []
    for key, value in centroid_dictionary.iteritems():
        al_value = value[2]/float(num_datapoints)
        al_fixed = al_value+.25
        alpha.insert(key,al_fixed)
    
    #try if completed_rides is populated
    try:
        completed_lats = list(completed_rides['start_location_lat'])
        completed_longs = list(completed_rides['start_location_long'])
        completed_source = ColumnDataSource( data=dict(
            lat=completed_lats,
            lon=completed_longs,
    )
)
        completed_dots = Circle(x="lon", y="lat", size=15, fill_color="green", fill_alpha=0.1, line_color=None)
        plot.add_glyph(completed_source, completed_dots)
    except:
        pass
    
    #try if unfulfilled_rides is populated
    try:
        unfulfilled_lats = list(unfulfilled_rides['start_location_lat'])
        unfulfilled_longs = list(unfulfilled_rides['start_location_long'])
        unfulfilled_source = ColumnDataSource(
        data=dict(
            lat=unfulfilled_lats,
            lon=unfulfilled_longs,

        )
    )
        unfulfilled_dots = Circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, line_color=None)
        plot.add_glyph(unfulfilled_source, unfulfilled_dots)
    except:
        pass
    #creating centroid source and circle
    centroidlats = centroids[:,0]
    centroidlongs = centroids[:,1]
    centroid_source = ColumnDataSource(
        data=dict(
            lat=centroidlats, 
            lon=centroidlongs,
             alpha=alpha
        )
    )
    centroid_dots = Circle(x="lon", y="lat", size=45, fill_color='#8B008B', fill_alpha='alpha', line_color=None)
    plot.add_glyph(centroid_source, centroid_dots)
    
    #finishing the plot
    plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
    show(plot)

In [124]:
dd['lat']= cents[:,0]
dd["long"] = cents[:,1]

In [127]:
dd.to_csv('current_hotspots.csv')

In [134]:
neighborhoods = {'nSoco':[[30.256315, -97.745302],[30.243497, -97.753048]],
                'sSoco':[[30.239975, -97.750773],[30.227151, -97.762931]],
                'nSoLamar': [[30.262691, -97.755906],[30.251125, -97.766893]],
                'midSoLamar': [[30.250847, -97.765198],[30.245926, -97.777751]],
                'sSoLamar':[[30.245926, -97.777751],[30.234609, -97.795969]],
                'Downtown':[[30.283521,-97.730909],[30.261218, -97.762784]],
                'Guadeloupe':[[30.297992, -97.740339],[30.281659, -97.743150]],
                'Domain':[[30.406832, -97.715588],[30.393294, -97.730930]],
                'Airport':[[30.215680, -97.659969],[30.199964,-97.674904]]}


In [132]:
for i in neighborhoods:
    print neighborhoods[i][0][0]

30.261218
30.406832
30.281659
30.245926
30.256315
30.250847
30.239975
30.262691
30.21568


In [158]:
def neighborhood(row):
    nbhd = 'None'
    for area in neighborhoods:
        lat = row['start_location_lat']
        lon = row['start_location_long']
        if lat <= neighborhoods[area][0][0] and lat >= neighborhoods[area][1][0]\
         and lon <= neighborhoods[area][0][1] and lon >= neighborhoods[area][1][1]:
            nbhd = area
    return nbhd

        

In [149]:
neighborhoods['Downtown'][0][1]

-97.730909

In [142]:
for x in d1.loc[:,'start_location_lat']:
    if x <= neighborhoods['Downtown'][0][0] and x >= neighborhoods['Downtown'][0][1]:
        print x

30.268438
30.214381
30.271621
30.23698
30.281789
30.235719
30.274565
30.2332
30.246493
30.251277
30.281789
30.24056
30.1813927587
30.246493
30.282725
30.274565
30.199004
30.255466
30.235216
30.264752
30.274565
30.228511
30.227585
30.282725
30.227264
30.282725
30.2077181442
30.274565
30.209257
30.235087
30.247345
30.247345
30.245261
30.282725
30.264882
30.247345
30.247345
30.247345
30.247345
30.197429
30.247345
30.237755
30.247345
30.210107
30.26022
30.247345
30.223204
30.247345
30.210107
30.247345
30.235649
30.282374
30.210014
30.247345
30.260505
30.260568
30.229004
30.267775
30.268501
30.247345
30.247345
30.244566
30.235649
30.267983
30.258413
30.235649
30.260829
30.266616
30.282374
30.210014
30.247345
30.247345
30.264524
30.235442
30.282534
30.2021489
30.264527
30.267492
30.265339
30.264737
30.262063
30.235442
30.25613
30.266616
30.269084
30.210047
30.267101
30.268771
30.235442
30.246493
30.269084
30.1974292
30.280847
30.235266
30.235266
30.258035
30.247047
30.235266
30.269084
30.235

In [159]:
d1['area'] = d1.apply(neighborhood,axis=1)

In [241]:
dr = d1[d1['area']!='None']

In [244]:
fare = dr[['total_fare','area']]

In [246]:
fare = fare[fare['total_fare']!= 0.0]

In [257]:
drr['avg_fare'] = fare.groupby(['area'])['total_fare'].mean().values

In [259]:
drr.to_csv('current_areas.csv')

In [208]:
drr = pd.DataFrame(dr.groupby(['area'], as_index=False)['status_value'].mean())

In [216]:
drr['total_rides'] = (dr.groupby(['area'],as_index=False)['start_location_lat'].count() )['start_location_lat']

In [218]:
drr['avg_fare'] = (dr.groupby(['area'],as_index=False)['total_fare'].mean())['total_fare']

In [220]:
def match_lat_longs(row):
    for area in neighborhoods:
         if row['area'] == area:
            return neighborhoods[area]

In [222]:
drr['area_coordinates']=drr.apply(match_lat_longs,axis=1)

In [260]:
drr

,area,status_value,total_rides,avg_fare,area_coordinates
0,Airport,1.000000,15,21.761333,"[[30.21568, -97.659969], [30.199964, -97.674904]]"
1,Domain,0.800000,10,17.403750,"[[30.406832, -97.715588], [30.393294, -97.73093]]"
2,Downtown,0.493776,241,11.896549,"[[30.283521, -97.730909], [30.261218, -97.7627..."
3,Guadeloupe,0.226667,75,13.516429,"[[30.297992, -97.740339], [30.281659, -97.74315]]"
4,midSoLamar,1.000000,5,8.334000,"[[30.250847, -97.765198], [30.245926, -97.7777..."
5,nSoLamar,0.281250,32,19.110000,"[[30.262691, -97.755906], [30.251125, -97.7668..."
6,nSoco,0.666667,33,11.276818,"[[30.256315, -97.745302], [30.243497, -97.7530..."
7,sSoLamar,0.208333,48,9.271000,"[[30.245926, -97.777751], [30.234609, -97.7959..."
8,sSoco,0.571429,21,10.881667,"[[30.239975, -97.750773], [30.227151, -97.7629..."


In [224]:
drr.to_csv('current_areas.csv')

In [228]:
future_hotspots = pd.DataFrame(cents, columns=[['lat','long']])

In [229]:
future_hotspots.to_csv('future_hotspots.csv')

In [475]:
cents

array([[ 30.27791298, -97.74584305],
       [ 30.23335194, -97.75710345],
       [ 30.31067603, -97.71648712],
       [ 30.39049817, -97.73950843],
       [ 30.31676449, -97.84064379]])

In [265]:
areadf = df.copy()
areadf['area'] = areadf.apply(neighborhood,axis=1)


In [266]:
areadf

,start_location_lat,start_location_long,created_date,tod,status,total_fare,area
0,38.676306,-121.038976,2016-06-03 21:14:40,21:14:40,NO_AVAILABLE_DRIVER,NaN,None
1,38.676837,-121.037948,2016-06-03 22:11:31,22:11:31,NO_AVAILABLE_DRIVER,NaN,None
2,30.324098,-97.728592,2016-06-04 03:23:12,03:23:12,NO_AVAILABLE_DRIVER,NaN,None
3,30.316335,-97.732381,2016-06-04 03:31:04,03:31:04,NO_AVAILABLE_DRIVER,NaN,None
4,30.280125,-97.770152,2016-06-04 03:40:23,03:40:23,NO_AVAILABLE_DRIVER,NaN,None
5,30.271075,-97.725108,2016-06-04 03:55:27,03:55:27,NO_AVAILABLE_DRIVER,NaN,None
6,30.271075,-97.725108,2016-06-04 03:55:57,03:55:57,NO_AVAILABLE_DRIVER,NaN,None
7,30.269378,-97.746428,2016-06-04 04:11:10,04:11:10,NO_AVAILABLE_DRIVER,NaN,Downtown
8,30.240889,-97.722964,2016-06-04 04:11:24,04:11:24,NO_AVAILABLE_DRIVER,NaN,None
9,30.269860,-97.746534,2016-06-04 04:16:17,04:16:17,NO_AVAILABLE_DRIVER,NaN,Downtown


In [352]:
def prep_df(dataframe):
        '''
        takes in dataframe of raw data and cleans it so that it can be passed
        into my regression model to predict n_rides for the next half-hour block.

        input:
        -dataframe

        output:
        -X dataframe
        -y dataframe
        '''
        data = dataframe.copy()
        data['month']= data['created_date'].dt.month
        data['day'] = data['created_date'].dt.day
        data['day_of_week'] = data['created_date'].dt.weekday
        # creating a COUNT response variable
        data = data.groupby(['day_of_week','month','period','day','area'])['start_location_lat'].count().reset_index(name="count")
        #data = pd.get_dummies(data, columns=['day_of_week'], drop_first = True)
        y = data.pop('count')
        data.pop('area')
        X = data
        return X,y

In [269]:
areadf['period'] = areadf.apply(hotspot.period,axis=1)

In [283]:
# areadf = areadf[areadf.loc[:,'area']!='None']
areadf['area'].unique()

array(['Downtown', 'Domain', 'nSoco', 'nSoLamar', 'Airport', 'Guadeloupe',
       'sSoLamar', 'midSoLamar', 'sSoco'], dtype=object)

In [364]:
airportdf = areadf[areadf.loc[:,'area']=='Airport']
Guadaloupe = areadf[areadf.loc[:,'area']=='Guadeloupe']
sSoLamar = areadf[areadf.loc[:,'area']=='sSoLamar']
midSoLamar = areadf[areadf.loc[:,'area']=='midSoLamar']
nSoLamar = areadf[areadf.loc[:,'area']=='nSoLamar']
nSoco = areadf[areadf.loc[:,'area']=='nSoco']
sSoco = areadf[areadf.loc[:,'area']=='sSoco']
Domain = areadf[areadf.loc[:,'area']=='Domain']
Downtown = areadf[areadf.loc[:,'area']=='Downtown']

In [436]:
dataframes = {'airportdf':airportdf,'Guadaloupe':Guadaloupe,
              'sSoLamar':sSoLamar,'midSoLamar':midSoLamar,'nSoLamar':nSoLamar,'nSoco':nSoco,'sSoco':sSoco,
              'Domain':Domain,'Downtown':Downtown}

In [337]:
date = pd.to_datetime('2016-10-09 22:00:00')

In [338]:
from sklearn.linear_model import LinearRegression
mod = LinearRegression()

In [429]:
X,y = hotspot.split_at_date(Domain,date)
X_train,y_train = prep_df(X)
X_test, y_test = prep_df(y)
std = np.var(y_train)**.5

In [430]:
mod.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [431]:
# X_test = pd.get_dummies(columns=['day_of_week','month','period','day'],data=X_test)
low_estimate = mod.predict(X_test)-std
high_estimate = mod.predict(X_test)+std
print int(round(low_estimate)),int(round(high_estimate))

1 5


In [439]:
dictionary = {}
for x in dataframes:
    Xd,yd = hotspot.split_at_date(dataframes[x],date)
    before_train,after_train = prep_df(Xd)
    print x,len(before_train)
    before_test, after_test = prep_df(yd)
    mod.fit(before_train,after_train)
    prediction = mod.predict(before_test)
    dictionary[str(x)] = prediction

Downtown 4747
Domain 1309
sSoLamar 1372
nSoco 2211
Guadaloupe 1926
midSoLamar 713
sSoco 1202
nSoLamar 1957
airportdf 2165


In [449]:
lst = []
for score in dictionary:
    Xd,yd = hotspot.split_at_date(dataframes[score],date)
    before_train,after_train = prep_df(Xd)
    y_previous = int(after_train.tail(1))
    print y_previous
    if dictionary[score] > y_previous:
        lst.append((score,1))
    else:
        lst.append((score,0))

2
2
6
21
3
2
2
14
4


In [473]:
d2.groupby(['cluster'])['start_location_long'].count()

cluster
0     63
1      1
2      7
3    300
4     40
Name: start_location_long, dtype: int64